# Assignment 4.1: Feature Store - Exercise
Sinthuja Bates

---

## Notebook Setup

In [2]:
# install sagemaker
!pip3 install -U sagemaker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 7.0 MB/s eta 0:00:00:00:01
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 5.3.1
    Uninstalling PyYAML-5.3.1:
      Successfully uninstalled PyYAML-5.3.1
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.29.0
    Uninstalling sagemaker-2.29.0:
      Successfully uninstalled sagemaker-2.29.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.18.216 requires botocore==1.19.56, but you have botocore 1.34.113 which is incompatible.
awscli 1.18.216 requires PyYAML<5.4,>=3.10; python_version != "3.4", but you have pyyaml 6.0.1 which is incompatible.
awscli 1.18.216 requires s3transfer<0.4.0,>=0.3.0, but you have s3transfer 0.10.1 which is incompatible.


In [3]:
# set up sagemaker with role and bucket
import os
import boto3
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name

bucket = sess.default_bucket()
prefix = "DEMO-breast-cancer-prediction-xgboost-highlevel"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


---

## Data preparation
**Data Sources**

Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

Breast Cancer Wisconsin (Diagnostic) Data Set [https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)].

_Also see:_ Breast Cancer Wisconsin (Diagnostic) Data Set [https://www.kaggle.com/uciml/breast-cancer-wisconsin-data].

In [6]:
# download the data and save it in the local folder with the name data.csv and take a look at it
import pandas as pd
import numpy as np

s3 = boto3.client("s3")

filename = "wdbc.csv"
s3.download_file(
    f"sagemaker-example-files-prod-{region}", "datasets/tabular/breast_cancer/wdbc.csv", filename
)
data = pd.read_csv(filename, header=None)

# specify columns extracted from wbdc.names
data.columns = [
    "id",
    "diagnosis",
    "radius_mean",
    "texture_mean",
    "perimeter_mean",
    "area_mean",
    "smoothness_mean",
    "compactness_mean",
    "concavity_mean",
    "concave points_mean",
    "symmetry_mean",
    "fractal_dimension_mean",
    "radius_se",
    "texture_se",
    "perimeter_se",
    "area_se",
    "smoothness_se",
    "compactness_se",
    "concavity_se",
    "concave points_se",
    "symmetry_se",
    "fractal_dimension_se",
    "radius_worst",
    "texture_worst",
    "perimeter_worst",
    "area_worst",
    "smoothness_worst",
    "compactness_worst",
    "concavity_worst",
    "concave points_worst",
    "symmetry_worst",
    "fractal_dimension_worst",
]

# save the data
data.to_csv("data.csv", sep=",", index=False)

data.sample(8)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
404,904969,B,12.340,14.95,78.29,469.1,0.08682,0.04571,0.021090,0.020540,...,13.180,16.85,84.11,533.1,0.1048,0.06744,0.04921,0.04793,0.2298,0.05974
395,903811,B,14.060,17.18,89.75,609.1,0.08045,0.05361,0.026810,0.032510,...,14.920,25.34,96.42,684.5,0.1066,0.12310,0.08460,0.07911,0.2523,0.06609
187,874373,B,11.710,17.19,74.68,420.3,0.09774,0.06141,0.038090,0.032390,...,13.010,21.39,84.42,521.5,0.1323,0.10400,0.15210,0.10990,0.2572,0.07097
117,864729,M,14.870,16.67,98.64,682.5,0.11620,0.16490,0.169000,0.089230,...,18.810,27.37,127.10,1095.0,0.1878,0.44800,0.47040,0.20270,0.3585,0.10650
412,905539,B,9.397,21.68,59.75,268.8,0.07969,0.06053,0.037350,0.005128,...,9.965,27.99,66.61,301.0,0.1086,0.18870,0.18680,0.02564,0.2376,0.09206
334,897374,B,12.300,19.02,77.88,464.4,0.08313,0.04202,0.007756,0.008535,...,13.350,28.46,84.53,544.3,0.1222,0.09052,0.03619,0.03983,0.2554,0.07207
503,915143,M,23.090,19.83,152.10,1682.0,0.09342,0.12750,0.167600,0.100300,...,30.790,23.87,211.50,2782.0,0.1199,0.36250,0.37940,0.22640,0.2908,0.07277
400,90439701,M,17.910,21.02,124.40,994.0,0.12300,0.25760,0.318900,0.119800,...,20.800,27.78,149.60,1304.0,0.1873,0.59170,0.90340,0.19640,0.3245,0.11980


In [5]:
# replace the M/B diagnosis with a 1/0 boolean value. 
data["diagnosis"] = data["diagnosis"].apply(lambda x: ((x == "M")) + 0)
data.sample(8)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
560,925292,0,14.050,27.15,91.38,600.4,0.09929,0.11260,0.04462,0.043040,...,15.30,33.17,100.20,706.7,0.1241,0.22640,0.13260,0.10480,0.2250,0.08321
524,917897,0,9.847,15.68,63.00,293.2,0.09492,0.08419,0.02330,0.024160,...,11.24,22.99,74.32,376.5,0.1419,0.22430,0.08434,0.06528,0.2502,0.09209
313,893988,0,11.540,10.72,73.73,409.1,0.08597,0.05969,0.01367,0.008907,...,12.34,12.87,81.23,467.8,0.1092,0.16260,0.08324,0.04715,0.3390,0.07434
400,90439701,1,17.910,21.02,124.40,994.0,0.12300,0.25760,0.31890,0.119800,...,20.80,27.78,149.60,1304.0,0.1873,0.59170,0.90340,0.19640,0.3245,0.11980
241,883539,0,12.420,15.04,78.61,476.5,0.07926,0.03393,0.01053,0.011080,...,13.20,20.37,83.85,543.4,0.1037,0.07776,0.06243,0.04052,0.2901,0.06783
41,855563,1,10.950,21.35,71.90,371.1,0.12270,0.12180,0.10440,0.056690,...,12.84,35.34,87.22,514.0,0.1909,0.26980,0.40230,0.14240,0.2964,0.09606
510,91550,0,11.740,14.69,76.31,426.0,0.08099,0.09661,0.06726,0.026390,...,12.45,17.60,81.25,473.8,0.1073,0.27930,0.26900,0.10560,0.2604,0.09879
89,861598,0,14.640,15.24,95.77,651.9,0.11320,0.13390,0.09966,0.070640,...,16.34,18.24,109.40,803.6,0.1277,0.30890,0.26040,0.13970,0.3151,0.08473


In [7]:
# data split in three sets, training, validation and batch inference
rand_split = np.random.rand(len(data))
train_list = rand_split < 0.8
val_list = (rand_split >= 0.8) & (rand_split < 0.9)
batch_list = rand_split >= 0.9

data_train = data[train_list].drop(["id"], axis=1)
data_val = data[val_list].drop(["id"], axis=1)
data_batch = data[batch_list].drop(["diagnosis"], axis=1)
data_batch_noID = data_batch.drop(["id"], axis=1)

In [8]:
upload those data sets in S3
train_file = "train_data.csv"
data_train.to_csv(train_file, index=False, header=False)
sess.upload_data(train_file, key_prefix="{}/train".format(prefix))

validation_file = "validation_data.csv"
data_val.to_csv(validation_file, index=False, header=False)
sess.upload_data(validation_file, key_prefix="{}/validation".format(prefix))

batch_file = "batch_data.csv"
data_batch.to_csv(batch_file, index=False, header=False)
sess.upload_data(batch_file, key_prefix="{}/batch".format(prefix))

batch_file_noID = "batch_data_noID.csv"
data_batch_noID.to_csv(batch_file_noID, index=False, header=False)
sess.upload_data(batch_file_noID, key_prefix="{}/batch".format(prefix))

's3://sagemaker-us-east-1-711667138246/DEMO-breast-cancer-prediction-xgboost-highlevel/batch/batch_data_noID.csv'

---

## Training job and model creation
The below cell uses the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) to kick off the training job using both our training set and validation set. Not that the objective is set to 'binary:logistic' which trains a model to output a probability between 0 and 1 (here the probability of a tumor being malignant).

In [9]:
%%time
from time import gmtime, strftime

job_name = "xgb-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
output_location = "s3://{}/{}/output/{}".format(bucket, prefix, job_name)
image = sagemaker.image_uris.retrieve(
    framework="xgboost", region=boto3.Session().region_name, version="1.7-1"
)

sm_estimator = sagemaker.estimator.Estimator(
    image,
    role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size=50,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sess,
)

sm_estimator.set_hyperparameters(
    objective="binary:logistic",
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    verbosity=0,
    num_round=100,
)

train_data = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train".format(bucket, prefix),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
validation_data = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validation".format(bucket, prefix),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
data_channels = {"train": train_data, "validation": validation_data}

# Start training by calling the fit method in the estimator
sm_estimator.fit(inputs=data_channels, job_name=job_name, logs=True)

INFO:sagemaker:Creating training-job with name: xgb-2024-06-01-16-26-21


2024-06-01 16:26:22 Starting - Starting the training job...
2024-06-01 16:26:36 Starting - Preparing the instances for training...
2024-06-01 16:27:08 Downloading - Downloading input data...
2024-06-01 16:27:33 Downloading - Downloading the training image...
2024-06-01 16:28:13 Training - Training image download completed. Training in progress...[2024-06-01 16:28:24.364 ip-10-0-221-110.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-06-01 16:28:24.386 ip-10-0-221-110.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-06-01:16:28:24:INFO] Imported framework sagemaker_xgboost_container.training
[2024-06-01:16:28:24:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2024-06-01:16:28:24:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:16:28:24:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-06-01:16:28:24:INFO] Determined 0 GPU(s) available on the inst

---

## Batch Transform

In SageMaker Batch Transform, we introduced 3 new attributes - __input_filter__, __join_source__ and __output_filter__. In the below cell, we use the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) to kick-off several Batch Transform jobs using different configurations of these 3 new attributes. Please refer to [this page](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform-data-processing.html) to learn more about how to use them.




#### 1. Create a transform job with the default configurations
Let's first skip these 3 new attributes and inspect the inference results. We'll use it as a baseline to compare to the results with data processing.

In [10]:
%%time

sm_transformer = sm_estimator.transformer(1, "ml.m4.xlarge")

# start a transform job
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file_noID
)  # use input data without ID column
sm_transformer.transform(input_location, content_type="text/csv", split_type="Line")
sm_transformer.wait()

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-06-01-16-29-16-090
INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-06-01-16-29-16-836


..........................................
[2024-06-01:16:36:12:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:16:36:12:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:16:36:12:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
[2024-06-01:16:36:12:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:16:36:12:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:16:36:12:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(pin

Let's inspect the output of the Batch Transform job in S3. It should show the list probabilities of tumors being malignant.

In [11]:
import re


def get_csv_output_from_s3(s3uri, batch_file):
    file_name = "{}.out".format(batch_file)
    match = re.match("s3://([^/]+)/(.*)", "{}/{}".format(s3uri, file_name))
    output_bucket, output_prefix = match.group(1), match.group(2)
    s3.download_file(output_bucket, output_prefix, file_name)
    return pd.read_csv(file_name, sep=",", header=None)

In [12]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file_noID)
output_df.head(8)

,0
0,0.015076
1,0.015076
2,0.015076
3,0.015076
4,0.015076
5,0.015076
6,0.015076
7,0.015076


#### 2. Join the input and the prediction results 
Now, let's associate the prediction results with their corresponding input records. We can also use the __input_filter__ to exclude the ID column easily and there's no need to have a separate file in S3.

* Set __input_filter__ to "$[1:]": indicates that we are excluding column 0 (the 'ID') before processing the inferences and keeping everything from column 1 to the last column (all the features or predictors)  
  
  
* Set __join_source__ to "Input": indicates our desire to join the input data with the inference results  

* Leave __output_filter__ to default ('$'), indicating that the joined input and inference results be will saved as output.

In [13]:
# content_type / accept and split_type / assemble_with are required to use IO joining feature
sm_transformer.assemble_with = "Line"
sm_transformer.accept = "text/csv"

# start a transform job
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file
)  # use input data with ID column cause InputFilter will filter it out
sm_transformer.transform(
    input_location,
    split_type="Line",
    content_type="text/csv",
    input_filter="$[1:]",
    join_source="Input",
)
sm_transformer.wait()

INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-06-01-16-38-16-847


........................................[2024-06-01:16:45:01:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:16:45:01:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:16:45:01:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      return 404 "{}";
    }
  }
}
[2024-06-01 16:45:01 +0000] [19] [INFO] Star

Let's inspect the output of the Batch Transform job in S3. It should show the list of tumors identified by their original feature columns and their corresponding probabilities of being malignant.

In [14]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file)
output_df.head(8)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,84358402,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.19800,0.10430,0.1809,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0.015076
1,853201,17.57,15.05,115.00,955.1,0.09847,0.1157,0.09875,0.07953,0.1739,...,19.52,134.90,1227.0,0.1255,0.2812,0.2489,0.1456,0.2756,0.07919,0.015076
2,854253,16.74,21.59,110.10,869.5,0.09610,0.1336,0.13480,0.06018,0.1896,...,29.02,133.50,1229.0,0.1563,0.3835,0.5409,0.1813,0.4863,0.08633,0.015076
3,855563,10.95,21.35,71.90,371.1,0.12270,0.1218,0.10440,0.05669,0.1895,...,35.34,87.22,514.0,0.1909,0.2698,0.4023,0.1424,0.2964,0.09606,0.015076
4,856106,13.28,20.28,87.32,545.2,0.10410,0.1436,0.09847,0.06158,0.1974,...,28.00,113.10,907.2,0.1530,0.3724,0.3664,0.1492,0.3739,0.10270,0.015076
5,85715,13.17,18.66,85.98,534.6,0.11580,0.1231,0.12260,0.07340,0.2128,...,27.95,102.80,759.4,0.1786,0.4166,0.5006,0.2088,0.3900,0.11790,0.015076
6,859283,14.78,23.94,97.40,668.3,0.11720,0.1479,0.12670,0.09029,0.1953,...,33.39,114.60,925.1,0.1648,0.3416,0.3024,0.1614,0.3321,0.08911,0.015076
7,859983,13.80,15.79,90.43,584.1,0.10070,0.1280,0.07789,0.05069,0.1662,...,20.86,110.30,812.4,0.1411,0.3542,0.2779,0.1383,0.2589,0.10300,0.015076


#### 3. Update the output filter to keep only ID and prediction results
Let's change __output_filter__ to "$[0,-1]", indicating that when presenting the output, we only want to keep column 0 (the 'ID') and the last column (the inference result i.e. the probability of a given tumor to be malignant)

In [15]:
# start another transform job
sm_transformer.transform(
    input_location,
    split_type="Line",
    content_type="text/csv",
    input_filter="$[1:]",
    join_source="Input",
    output_filter="$[0,-1]",
)
sm_transformer.wait()

INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-06-01-16-46-32-812


...........................................[2024-06-01:16:53:40:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:16:53:40:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:16:53:40:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:16:53:40:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
  

Now, let's inspect the output of the Batch Transform job in S3 again. It should show 2 columns: the ID and their corresponding probabilities of being malignant.

In [16]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file)
output_df.head(8)

,0,1
0,84358402,0.015076
1,853201,0.015076
2,854253,0.015076
3,855563,0.015076
4,856106,0.015076
5,85715,0.015076
6,859283,0.015076
7,859983,0.015076


create_model(role=role, image_uri=XGBOOST_IMAGE)In summary, we can use newly introduced 3 attributes - __input_filter__, __join_source__, __output_filter__ to 
1. Filter / select useful features from the input dataset. e.g. exclude ID columns.
2. Associate the prediction results with their corresponding input records.
3. Filter the original or joined results before saving to S3. e.g. keep ID and probability columns only.

## Upload the Sagemaker Model created during our training job to the Sagemaker Model Registry

In [19]:
sagemaker = boto3.client("sagemaker")

model_name = job_name
print(model_name)


info = sagemaker.describe_training_job(TrainingJobName=model_name)
model_data = info["ModelArtifacts"]["S3ModelArtifacts"]

primary_container = {"Image": image, "ModelDataUrl": model_data}

# Save our model to the Sagemaker Model Registry
create_model_response = sagemaker.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

xgb-2024-06-01-16-26-21
arn:aws:sagemaker:us-east-1:711667138246:model/xgb-2024-06-01-16-26-21


In [20]:
# Inspect Training Job Details
info

{'TrainingJobName': 'xgb-2024-06-01-16-26-21',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:711667138246:training-job/xgb-2024-06-01-16-26-21',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-711667138246/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-06-01-16-26-21/xgb-2024-06-01-16-26-21/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'eta': '0.2',
  'gamma': '4',
  'max_depth': '5',
  'min_child_weight': '6',
  'num_round': '100',
  'objective': 'binary:logistic',
  'subsample': '0.8',
  'verbosity': '0'},
 'AlgorithmSpecification': {'TrainingImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1',
  'TrainingInputMode': 'File',
  'MetricDefinitions': [{'Name': 'train:mae',
    'Regex': '.*\\[[0-9]+\\].*#011train-mae:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
   {'Name': 'validation:aucpr',
    'Regex': '.*\\[[0-9]+\\].*#011validation-aucpr:([-+]?[0-9]*\

In [21]:
# Create Endpoint Configuration


# Create an endpoint config name. Here we create one based on the date  
# so it we can search endpoints based on creation time.
endpoint_config_name = 'lab4-1-endpoint-config' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())                            
                            
instance_type = 'ml.m5.xlarge'

endpoint_config_response = sagemaker.create_endpoint_config(
    EndpointConfigName=endpoint_config_name, # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name, 
            "InstanceType": instance_type, # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ]
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")


Created EndpointConfig: arn:aws:sagemaker:us-east-1:711667138246:endpoint-config/lab4-1-endpoint-config2024-06-01-17-02-46


In [22]:
# Deploy our model to real-time endpoint

endpoint_name = 'lab4-1-endpoint' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())                            


create_endpoint_response = sagemaker.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name) 

In [25]:
# Wait for endpoint to spin up
from time import sleep

sagemaker.describe_endpoint(EndpointName=endpoint_name)

while True:
    print("Getting Job Status")
    res = sagemaker.describe_endpoint(EndpointName=endpoint_name)
    state = res["EndpointStatus"]
    
    if state == "InService":
        print("Endpoint in Service")
        break
    elif state == "Creating":
        print("Endpoint still creating...")
        sleep(60)
    else:
        print("Endpoint Creation Error - Check Sagemaker Console")
        break

Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint in Service


In [26]:
# Invoke Endpoint

sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

response = sagemaker_runtime.invoke_endpoint(
                            EndpointName=endpoint_name,
                            ContentType='text/csv',
                            Body=data_batch_noID.to_csv(header=None, index=False).strip('\n').split('\n')[0]
                            )
print(response['Body'].read().decode('utf-8'))

0.015076400712132454



In [27]:
# Examine Response Body

response

{'ResponseMetadata': {'RequestId': '9e414efb-561c-4310-bd9b-aeb5f6527055',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9e414efb-561c-4310-bd9b-aeb5f6527055',
   'x-amzn-invoked-production-variant': 'variant1',
   'date': 'Sat, 01 Jun 2024 17:08:18 GMT',
   'content-type': 'text/csv; charset=utf-8',
   'content-length': '21',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'variant1',
 'Body': <botocore.response.StreamingBody at 0x7f5d0e591150>}

In [45]:
# Delete Endpoint

sagemaker.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'c2b381e2-33a0-4b70-8732-fbaa44fab5a0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c2b381e2-33a0-4b70-8732-fbaa44fab5a0',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 22 Aug 2023 07:10:18 GMT'},
  'RetryAttempts': 0}}